In [3]:
import sys
import glob
import re
import fnmatch
from os import listdir
from os.path import join, isfile, basename

import itertools

import numpy as np
from numpy import float32, int32, uint8, dtype, genfromtxt

import pandas as pd

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
# VFB labels and label names
labels = [16,64,8,32,2,4,65,66,33,67,34,17,69,70,35,71,9,18,72,36,73,74,37,75,19,76,38,77,39,78,79,20,5,40,80,10,81,82,83,84,85,86,11,22,23,24,12,3,6,49,50,25,51,13,52,26,53,27,54,55,56,28,7,14,57,58,29,59,30,60,15,61,31,62,63]
label_names_file = '/groups/saalfeld/home/bogovicj/vfb/DrosAdultBRAINdomains/refData/Original_Index.tsv'

label_names = pd.read_csv( label_names_file, delimiter='\t', header=0 )
# print label_names[ label_names['Stack id'] == 11 ]['JFRCtempate2010.mask130819' ].iloc[0]
# print label_names[ label_names['Stack id'] == 70 ]['JFRCtempate2010.mask130819' ].iloc[0]

def get_label_name( label_id ):
    return label_names[ label_names['Stack id'] == label_id ]['JFRCtempate2010.mask130819' ].iloc[0]

# label_names['JFRCtempate2010.mask130819']

label_shorthand_col ='JFRCtempate2010.mask130819'
label_id_col ='Stack id'


# Find left-right matching labels
rnames = label_names[ label_names.apply( lambda x : x[label_shorthand_col].endswith('_R'), axis=1 )]

lr_pair_list = []
for rn in rnames.loc[:,label_shorthand_col]:
    ln = rn.replace('_R','_L')
    id_R = label_names[ label_names[label_shorthand_col]==rn ].loc[:,label_id_col]
    id_L = label_names[ label_names[label_shorthand_col]==ln ].loc[:,label_id_col]
    lr_pair_list += [[id_R.values[0], id_L.values[0]]]
lr_pair_list = np.array( lr_pair_list )

In [5]:
path='/nrs/saalfeld/john/projects/flyChemStainAtlas/all_evals/F-antsFlip/cmtkCow/evalComp/stats0.txt'

def readlines( f ):
    f = open( f, 'r' )
    lines = f.readlines()
    f.close()
    return lines

def labelstat( lines, label, stat='MEDIAN'):
    statupper = stat.upper()
    for l in lines:
        if l.startswith(str(label)):
            if l.find(statupper) >= 0:
                return float( l.split()[2] )


In [6]:
base_dir = '/nrs/saalfeld/john/projects/flyChemStainAtlas/all_evals'

# templates = ['JFRCtemplate2010']
# reg_methods = [ 'cmtkCow', 'cmtkCOG', 'antsRegYang' ]

templates = ['F-antsFlip']
reg_methods = [ 'cmtkCow', 'cmtkCOG' ]


In [47]:
# parse all data into a data frame

expnamelist=[]
statlist=[]
datalist=[]
labellist=[]
linelist=[]

for template,reg in itertools.product( templates, reg_methods ):

    exp_dir = join( base_dir, template, reg )
#     print exp_dir
    expname = readlines( join(exp_dir,'name'))[0]
#     print expname 

    for line in [0,1,2,3]:
        eval_dir=join(exp_dir,'evalComp')
        
        # Read label stats
        statFile = '{}/stats{}.txt'.format(eval_dir,line)
        stat_lines = readlines( statFile )
        for sl in stat_lines:
            dat = sl.split()
            expnamelist += [ expname.replace('\n','') ]
            linelist += [ line ]
            labellist += [ int(dat[0]) ]
            statlist += [ dat[1] ]
            datalist += [ float(dat[2]) ]
            
        # Read total brain mask stats
        allStatFile = '{}/all_stats{}.txt'.format(eval_dir,line)
        all_stat_lines = readlines( allStatFile )
        for sl in all_stat_lines:
            if( sl.startswith('all ')):
                dat = sl.split()
                expnamelist += [ expname.replace('\n','') ]
                linelist += [ line ]
                labellist += [ -1 ] # -1 indicates total mask label
                statlist += [ dat[1] ]
                datalist += [ float(dat[2]) ]


dist_df = pd.DataFrame( {'EXP':expnamelist, 
                         'STAT':statlist,
                         'LINE':linelist,
                         'LABEL':labellist,
                         'VALUE':datalist })



In [52]:
# print dist_df
# print dist_df[ ((dist_df.STAT == 'MEDIAN') | (dist_df.STAT == 'COUNT')) & (dist_df.LABEL == 2) & (dist_df.LINE == 0) ]
# print ' '
# print dist_df[ ((dist_df.STAT == 'MEDIAN') | (dist_df.STAT == 'COUNT')) & (dist_df.LABEL == -1) & (dist_df.LINE == 0) ]

In [51]:
# Sort labels by average size

avg_counts = []

gb_line_label = dist_df[ (dist_df.STAT == 'COUNT') ].groupby(['LINE','LABEL'])
line_label_avgs = gb_line_label.mean().sort_values('VALUE', ascending=False)


